In [ ]:
import numpy, pandas, glob, random, seaborn, re, matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from matplotlib.lines import Line2D
from matplotlib import colors
import sklearn.linear_model
from sklearn.model_selection import KFold
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.api import anova_lm
from statsmodels.formula.api import ols
from statsmodels.stats import multitest
from pyplink import PyPlink
import rpy2.robjects as R
from utils import *
matplotlib.rcParams['figure.dpi'] = 300

In [3]:
# define cohort and taxonomy level
cohort = 'both'
taxa_level = 'genus'
bmi_or_not = 'bmi'
classification = 'silva'
classification_method = classification + '_classification'

# load phenotype 

In [5]:
# load pheno
micro_metab_unadj = pandas.read_csv('./joint_pheno/{0}_{1}_{2}_micro_metab_unadj.pheno'.format(cohort, taxa_level, classification), index_col = [0,1])
micro_metab_covar = pandas.read_csv('./joint_pheno/{0}_{1}_{2}_micro_metab_unadj.covar'.format(cohort, taxa_level, classification), index_col = [0,1])
pheno_covar_merged_full = micro_metab_unadj.join(micro_metab_covar)
pheno_covar_merged_full.index = pheno_covar_merged_full.index.droplevel(0)
pheno_covar_merged_full.columns = [rename_numerical_id(col) for col in pheno_covar_merged_full.columns]

# define covariates
q_covar = ['age','age_sq','sex_age','sex_age_sq','PC1','PC2','PC3','PC4','PC5', 'bmi']
c_covar = ['sex','batch']
rename_c_covar = ['C({0})'.format(covar) for covar in c_covar]
covariates = q_covar + c_covar

form_covar = ['scale(age)', 'I(scale(age)**2)', 'PC1','PC2','PC3','PC4','PC5', 'bmi']
int_covar = ['scale(age)*C(sex)', 'I(scale(age)**2)*C(sex)']
form_covar = form_covar + rename_c_covar + int_covar

# load association
micro_metab_assoc = pandas.read_csv('./assoc_results/{0}_sig_metab_{1}_{2}_classification_lasso.csv'.format(cohort, taxa_level, classification))
micro_metab_assoc['metabolite_id'] = [rename_numerical_id(identifier) for identifier in micro_metab_assoc['metabolite_id']]
micro_metab_assoc['tax_id'] = [rename_numerical_id(identifier) for identifier in micro_metab_assoc['tax_id']]
sig_metabolite = micro_metab_assoc['metabolite_id'].unique()
print(len(sig_metabolite))

# get metadata
metabolite_metadata = pandas.read_csv('./input_pheno/all_metab_11192021.pheno.metadata', sep = '\t', dtype=str)
metabolite_metadata = metabolite_metadata.groupby('metabolite_id').first()[['metabolite', 'sub_pathway', 'super_pathway']]
metabolite_metadata.index = [rename_numerical_id(identifier) for identifier in metabolite_metadata.index]
metabolite_metadata_dict = metabolite_metadata['metabolite'].to_dict()
metabolite_super_pathway_dict = metabolite_metadata['super_pathway'].to_dict()

taxa_metadata = pandas.read_csv('./joint_pheno/{0}_{1}.metadata'.format(taxa_level, classification_method), dtype=str, sep = '\t')
taxa_metadata = taxa_metadata.set_index('tax_id')
taxa_metadata.index = [rename_numerical_id(identifier) for identifier in taxa_metadata.index]
genus_metadata_dict = taxa_metadata['taxonomy'].str.split(';').str.get(-1).to_dict()

176


# get genotype

In [6]:
genotype = PyPlink('./input_bed/all_chr/all_genomes_09112019_all_chr')
# genotype = PyPlink('/homedir/notebooks-cheng/mgwas/data/snp_query_03182019/final_merged/bed_files/all_chr/merged_metabolome_inds_biallelic_variants_all_chr')
sample_ids = genotype.get_fam().iid.values

In [7]:
snp_list = 'validate_gwas/clumped_variants/metab/fastgwa/clumped_10perc_second_pass_list.txt'
sig_metab_assoc = pandas.read_csv(snp_list, sep = '\t', header = None)
sig_metab_assoc.columns = ['rsid', 'Phenotype']

# subset to phenotypes that have microbiome associations
sig_metab_assoc = sig_metab_assoc[sig_metab_assoc['Phenotype'].isin(micro_metab_assoc['metabolite_id'].values)]
sig_metab_assoc = sig_metab_assoc[~sig_metab_assoc['Phenotype'].str.contains('X99')]
print(len(sig_metab_assoc['Phenotype'].unique()))

176


# run GxE

In [8]:
from multiprocessing import Pool
from rich.progress import track
from itertools import product

pheno_covar_merged = pheno_covar_merged_full.copy()

gxe_results = []
rev_rsid_map = {}
microbes = list(micro_metab_assoc['tax_id'].unique())

# Assemble a table of all SNPs
metabs = list(sig_metab_assoc['Phenotype'].unique())
rsids = sig_metab_assoc['rsid'].unique()
renamed_rsids = pandas.Series({i: i.split(';')[0] for i in rsids})
dfs = [pandas.DataFrame(genotype.get_geno_marker(rsid), index = sample_ids, columns = [rsid]) for rsid in rsids]
geno = pandas.concat(dfs, axis=1)
geno.rename(columns = renamed_rsids, inplace=True)
geno_pheno_covar = pheno_covar_merged[metabs + microbes + covariates].join(geno, how = 'outer')

def betas(args):
    """Get the gene environment interaction coefficients."""
    metab, microbe = args
    rsids = renamed_rsids[sig_metab_assoc.loc[sig_metab_assoc['Phenotype'] == metab, 'rsid']].tolist()
    # print(metab, metabolite_metadata_dict[metab], f'# snps: {len(rsids)}', f'# bugs: {len(microbes)}')

    df = geno_pheno_covar[[metab] + [microbe] + rsids + covariates].dropna()

    gxes = []
    
    result = pandas.DataFrame(
        index = [metab + ':' + microbe + ':' + i for i in rsids],
        columns = ['metabolite_id','tax_id','rsid', 'genetics_beta', 'gxe_beta','pval', 
                   'baseline_rsq', 'without_microbe_rsq', 'gxe_rsq', 'n_alt_0', 'n_alt_1', 'n_alt_2']
    )
    for rsid in rsids:

        try:
            assert (df.groupby(rsid).count() > 30).all().all()
            n_alt_count = df.groupby(rsid).count()
            n_alt_0 = n_alt_count.loc[0][0]
            n_alt_1 = n_alt_count.loc[1][0]
            n_alt_2 = n_alt_count.loc[2][0]
        except:
            continue 
        gxe_term = rsid + ':' + microbe
        no_microbe_formula = metab + ' ~ ' + ' + '.join(form_covar + [rsid])    
        no_gxe_formula = metab + ' ~ ' + ' + '.join(form_covar + [rsid] + [microbe])
        gxe_formula = metab + ' ~ ' + ' + '.join(form_covar + [rsid] + [microbe] + [gxe_term])
        
        no_int_out = ols(no_gxe_formula, df).fit()
        baseline_rsq = no_int_out.rsquared
        
        no_microbe_rsq = ols(no_microbe_formula, df).fit().rsquared
        
        int_out = ols(gxe_formula, df).fit()
        int_rsq = int_out.rsquared
        gxe_beta = int_out.params.loc[gxe_term]
        genetics_beta = int_out.params.loc[rsid]
        gxe_pval = int_out.pvalues.loc[gxe_term]
        
        result.loc[metab + ':' + microbe + ':' + rsid] = [
            metab, microbe, rsid, genetics_beta, gxe_beta, gxe_pval, baseline_rsq, no_microbe_rsq, 
            int_rsq, n_alt_0, n_alt_1, n_alt_2
        ]
            
    return result

with Pool(8) as pool:
    args = [(met, mic) for met in metabs for mic in microbes]
    it = track(pool.imap_unordered(betas, args), total=len(args), description='Fitting models')
    results = pandas.concat(list(it))

fdr_pass, adj_pval, sidack, bonff = statsmodels.stats.multitest.multipletests(results['pval'].values, method='fdr_bh', alpha=0.05)
results['pass_fdr'] = adj_pval < 0.05
results['adj_pval'] = adj_pval


Output()

In [9]:
genes = rsid2gene(gxe_fold_results_annotated.rsid.unique())

NameError: name 'gxe_fold_results_annotated' is not defined

In [ ]:
gxe_fold_results_df = results.copy().dropna()
gxe_fold_results_annotated = gxe_fold_results_df.merge(metabolite_metadata, left_on = 'metabolite_id', right_index = True, how = 'left')
gxe_fold_results_annotated = gxe_fold_results_annotated.merge(taxa_metadata, left_on = 'tax_id', right_index = True, how = 'left')
gxe_fold_results_annotated['genus'] = gxe_fold_results_annotated['taxonomy'].str.split(';').str.get(-1)
gxe_fold_results_annotated['family'] = gxe_fold_results_annotated['taxonomy'].str.split(';').str.get(-2)

gxe_fold_results_annotated = gxe_fold_results_annotated.merge(genes, on = 'rsid', how = 'left')

gxe_fold_results_annotated.head()

In [ ]:
sig_gxe_fold_results_annotated = gxe_fold_results_annotated[gxe_fold_results_annotated['pass_fdr'] == True]

In [ ]:
min_count = 30
gxe_results_sig_abundant = sig_gxe_fold_results_annotated[(sig_gxe_fold_results_annotated['n_alt_0'] > min_count) & 
                                           (sig_gxe_fold_results_annotated['n_alt_1'] > min_count) & 
                                           (sig_gxe_fold_results_annotated['n_alt_2'] > min_count)]

In [ ]:
gxe_results_sig_abundant["gxe_only_rsq"] = gxe_results_sig_abundant.gxe_rsq - gxe_results_sig_abundant.baseline_rsq
temp_gxe_interest = gxe_results_sig_abundant.sort_values('pval', ascending=True).drop_duplicates()
temp_gxe_interest = temp_gxe_interest.reset_index(drop = True)

In [ ]:
from plotnine import *
import numpy as np
from scipy.stats import pearsonr

s = sig_gxe_fold_results_annotated

s.genetics_beta = s.genetics_beta.astype("float")
s.gxe_beta = s.gxe_beta.astype("float")
s.without_microbe_rsq = s.without_microbe_rsq.astype("float")
s.gxe_rsq = s.gxe_rsq.astype("float")
s.baseline_rsq = s.baseline_rsq.astype("float")
s["neg_log_pval"] = -np.log10(s.pval.astype("float"))
s["type"] = "opposite"
s.loc[s.gxe_beta * s.genetics_beta > 0, "type"] = "same"
s["gxe_only_rsq"] = s.gxe_rsq - s.baseline_rsq
s["genetics_rsq"] = s.without_microbe_rsq

pl = (
    ggplot(s, aes(y="gxe_only_rsq", x="genetics_rsq", color="type", group=1))
    + geom_smooth(linetype="dashed", method="lm")
    + geom_point(size=2)
    + theme_minimal()
    + geom_text(aes(label="metabolite"), data=s[s.gxe_only_rsq > 0.01], ha="left", nudge_x=0.01, size=10)
    + labs(x="R² genetics", y="R² gene:microbe", color="direction")
    + scale_color_manual(values=["tomato", "royalblue"])
)
print(pearsonr(s.gxe_only_rsq, s.genetics_rsq))
pl.save("figures/genetics_vs_gxe.pdf", width=6, height=5)
pl

In [ ]:
s[s.tax_id == "alpha_div"]

# plot

In [ ]:
for i in temp_gxe_interest.index[0:5]:
    hit = temp_gxe_interest.loc[i]
    sig_metab = hit['metabolite_id']
    sig_rsid = hit['rsid']
    sig_microbe = hit['tax_id']
    temp_hit_geno = pandas.DataFrame(genotype.get_acgt_geno_marker(sig_rsid), index = sample_ids, columns = [sig_rsid])
    temp_hit_geno = temp_hit_geno.join(pandas.DataFrame(genotype.get_geno_marker(sig_rsid), index = sample_ids, columns = [sig_rsid + '_alt_count']))
    temp_pheno_covar_merged = pheno_covar_merged[[sig_metab, sig_microbe] + covariates].join(temp_hit_geno, how = 'inner').dropna()

    bim = genotype.get_bim()
    hit_minor_allele = bim.loc[sig_rsid, 'a1']
    hit_major_allele = bim.loc[sig_rsid, 'a2']
    allele_order = [hit_major_allele*2, hit_minor_allele + hit_major_allele, hit_minor_allele*2]

    temp_cov_adj_formula = sig_metab + ' ~ ' + ' + '.join(form_covar + rename_c_covar)
    temp_cov_adj_lm = ols(temp_cov_adj_formula, temp_pheno_covar_merged).fit()
    temp_pheno_covar_merged['covar_adj'] = temp_cov_adj_lm.resid

    print(sig_rsid, metabolite_metadata_dict[sig_metab], genus_metadata_dict[sig_microbe])

    scatter_args = {'linewidth':0.3, 'edgecolor':'black', 'color':'slategrey'}
    line_args = {'color':'slategrey'}
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize = (10,4), sharey= False)
    sns.boxplot(x = sig_rsid, y= 'covar_adj', data = temp_pheno_covar_merged, 
                order = allele_order, ax = ax1, palette = 'Set1', showfliers = False, showcaps = False)
    sns.stripplot(x = sig_rsid, y= 'covar_adj', data = temp_pheno_covar_merged, 
                  order = allele_order, ax = ax1, palette = 'Set1', linewidth=0.5, alpha = 0.5)
    sns.regplot(x = sig_microbe, y = 'covar_adj', data = temp_pheno_covar_merged,
                scatter_kws = scatter_args, ax = ax2,
               line_kws = line_args)
    ax1.set_ylabel(metabolite_metadata_dict[sig_metab])
    ax2.set_ylabel('')
    ax2.set_xlabel(genus_metadata_dict[sig_microbe])
    sns.despine()
    plt.savefig(f"figures/{sig_metab}|{sig_microbe}|{sig_rsid}_basic.png", dpi=300, width=6, height=4, bbox_inches="tight")
    plt.show()
    plt.close()
    
    print(scipy.stats.pearsonr(temp_pheno_covar_merged['covar_adj'], temp_pheno_covar_merged[sig_microbe])[0])
    for index, (protein_genotype, subset_df) in enumerate(temp_pheno_covar_merged.groupby(sig_rsid)):
        slope, intercept, r, pval, std = scipy.stats.linregress(subset_df['covar_adj'], subset_df[sig_microbe])
        print(protein_genotype, r, pval)
    
    g = sns.FacetGrid(data=temp_pheno_covar_merged, col=sig_rsid, col_order = allele_order)
    g.map(sns.regplot, sig_microbe, sig_metab,  scatter_kws = scatter_args, line_kws = line_args)
    g.set(xlim=(temp_pheno_covar_merged[sig_microbe].min()*1.1, temp_pheno_covar_merged[sig_microbe].max()*1.1),
         ylim=(temp_pheno_covar_merged[sig_metab].min()*1.1, temp_pheno_covar_merged[sig_metab].max()*1.1))
    g.set_xlabels(genus_metadata_dict[sig_microbe])
    g.set_ylabels(metabolite_metadata_dict[sig_metab])
    plt.savefig(f"figures/{sig_metab}|{sig_microbe}|{sig_rsid}_groups.png", dpi=300, width=6, height=4, bbox_inches="tight")
    plt.show()
    plt.close()

In [ ]:
temp_pheno_covar_merged['affected'] = [1 if i.count(hit_minor_allele) > 0 
                                       else 0 
                                       for i in temp_pheno_covar_merged[sig_rsid]]
affected_order = [0, 1]
affected_x_tick_label = list(map({0:'unaffected', 1:'affected'}.get, affected_order))

print(ols('{0} ~ {1}'.format(sig_microbe,'affected'), data=temp_pheno_covar_merged).fit().pvalues['affected'])
print(ols('{0} ~ {1}'.format(sig_microbe,sig_rsid+'_alt_count'), data=temp_pheno_covar_merged).fit().pvalues[sig_rsid+'_alt_count'])


fig, ax = plt.subplots()
sns.boxplot(x = 'affected', y= sig_microbe, data = temp_pheno_covar_merged, order = [0, 1], ax = ax, palette = 'Set1')
sns.stripplot(x = 'affected', y= sig_microbe, data = temp_pheno_covar_merged, order = [0, 1], ax = ax, palette = 'Set1', linewidth=0.5)
ax.set_xticklabels(affected_x_tick_label)
ax.set_xlabel('')
ax.set_ylabel(genus_metadata_dict[sig_microbe])
plt.show()
plt.close()

In [ ]:
gxe_fold_results_annotated.shape

In [ ]:
(temp_gxe_interest.gxe_rsq - temp_gxe_interest.baseline_rsq).astype(float).describe()

In [ ]:
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests

def stars(p):
    if p>0.05:
        return "n.s."
    else:
        return sum(p < c for c in [0.05, 0.01, 0.001]) * "٭"

def enrichment(full_data, sig_data, column, figsize=(3, 4), min_sig=1):
    full_counts = full_data[column].value_counts()
    sig_counts = sig_data[column].value_counts()
    stats = pd.DataFrame({"full_counts": full_counts, "sig_counts": sig_counts, "n": full_counts.sum(), "sig_n": sig_counts.sum()}).fillna(0)
    stats["p"] = stats.apply(lambda df: hypergeom.sf(max(0, df.sig_counts - 1), df.n, max(df.full_counts, 1), df.sig_n), axis=1)
    stats["odds"] = (stats.sig_counts / stats.sig_n) / (stats.full_counts / stats.n)
    stats["log_odds"] = np.log(stats.odds)
    stats["q"] = multipletests(stats.p, method="fdr_bh")[1]
    
    stats["all"] = stats["full_counts"] / stats["n"]
    stats["significant"] = stats["sig_counts"] / stats["sig_n"]
    stats.index.name = column
    stats.reset_index(inplace=True)
    
    long = stats[stats.sig_counts >= min_sig].melt(id_vars=[column, "p", "q"], value_vars=["all", "significant"], value_name="prevalence", var_name="group")
    long[column] = pd.Categorical(long[column], long.sort_values(by="prevalence")[column].unique()) 
    long["sig_stars"] = long.q.apply(stars)
    
    pl = (
        ggplot(long, aes(x="prevalence", y=column, shape="group", color="group")) +
        geom_line(aes(group=column), color="black") +
        geom_point(size=2) +
        theme_minimal() +
        theme(figure_size=figsize) +
        labs(y="")
    )
    if (long.q < 0.05).any():
        pl += geom_text(
            aes(label="sig_stars"), 
            data=long[(long.q<0.05) & (long.group == "significant")], 
            color="black", va="center", ha="left", 
            nudge_x=(long.prevalence.max() - long.prevalence.min())*0.025)
    
    return stats.sort_values(by="p"), pl

In [ ]:
pathway_enr = enrichment(gxe_fold_results_annotated, sig_gxe_fold_results_annotated, "sub_pathway", figsize=(3, 5))
microbe_enr = enrichment(gxe_fold_results_annotated, sig_gxe_fold_results_annotated, "family", figsize=(3, 4))
gene_enr = enrichment(gxe_fold_results_annotated, sig_gxe_fold_results_annotated, "gene_symbol", figsize=(3, 7))

In [ ]:
pathway_enr[1].save("figures/enriched_pathways.pdf", width=3, height=5)
pathway_enr[1]

In [ ]:
microbe_enr[1].save("figures/enriched_microbes.pdf", width=3, height=4)
microbe_enr[1]

In [ ]:
gene_enr[1].save("figures/enriched_genes.pdf", width=3, height=7)
gene_enr[1]

In [ ]:
snp_enr[0]